# Función de _ranking_ 2: BM25F

Usaremos aquí la frecuencia de términos normalizada por campos ($ftf$, de _Field dependent normalized Term Frequency_). Así, para un término $t$ dado, y un campo $f \in \{url, header, body, title, anchor\}$ en el documento $d$, usaremos:

\begin{equation}
ftf_{d,f,t} = \frac{tf_{d,f,t}}{1 + B_f((\text{len}_{d,f} / \text{avlen}_f) - 1)}
\tag{1}
\end{equation}

Donde $tf_{d,f,t}$ es la frecuencia cruda de $t$ en el campo $f$ del documento $d$, $len_{d,f}$ es la longitud del campo $f$ en $d$, y $avlen_f$ es la longitud media del mismo campo en toda la colección.

Por supuesto, las correspondientes variables $avlen_{body}$, $avlen_{url}$, $avlen_{title}$, $avlen_{header}$ y $avlen_{anchor}$ se deberán computar usando de nuevo el conjunto de _training_. Los valores de $B_f$ serán parámetros adicionales dependientes de cada uno de los campos $f$, y al igual que los $c_f$ de la sección anterior, deberán ser ajustados. Si $avlen_f$ fuese cero, entonces definiremos $ftf_{d,f,t} = 0$ (si bien esto no debería ser necesario en este _dataset_ en concreto).

El peso global para el término $t$ en el documento $d$, usando ya todos los campos, sería:

\begin{equation}
w_{d,t} = \sum_{f} W_f \cdot ftf_{d,f,t}
\tag{2}
\end{equation}

Siendo, de nuevo, los distintos $W_f$ parámetros que determinan los pesos de importancia relativos dados a cada uno de los campos.

Puesto que, además, tenemos también una característica adicional no textual (el <b>pagerank</b>), la incorporaremos también en nuestra función de _ranking_ usando el método descrito en las transparencias de teoría.

En concreto, pues, y resumiendo, el _scoring_ global para el $d$ respecto a la consulta $q$ quedará definido como:

\begin{equation}
\sum_{t \in q} \frac{w_{d,t}}{K_1 + w_{d,t}}idf_t + \lambda V_{j}(f)
\tag{3}
\end{equation}

De nuevo aquí $K_1$ es un parámetro libre, y la función $V_{j}$ podría ser cualquier de las funciones logarítmicas, de saturación o sigmoide mencionadas en las transparencias de teoría, y que en este caso fijaremos simplemente como $V_{pagerank}(pr) = log(\lambda'+pr)$.

$\lambda$ y $\lambda'$ son los dos últimos parámetros libres adicionales para este modelo.

## Clase _BM25FScorer_

Definimos aquí la clase `BM25FScorer`, basada en la `CosineSimilarityScorer` anterior, pero incorporando en este caso todas las modificaciones necesarias para implementar la nueva funcionalidad descrita en los párrafos anteriores:

In [ ]:
class BM25FScorer(CosineSimilarityScorer):
    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None):
        super().__init__(idf, query_dict, params, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme)

        # Añadimos aquí los pesos ya específicos para BM25, y los nuevos parámetros libres...
        self.b_url = params['b_url']
        self.b_title = params['b_title']
        self.b_header = params['b_header']
        self.b_body_hits = params['b_body_hits']
        self.b_anchor = params['b_anchor']
        self.k1 = params['k1']
        self.pagerank_lambda = params['pagerank_lambda']
        self.pagerank_lambda_prime = params['pagerank_lambda_prime']

        # ... y aqui tres estructuras de datos adicionales, necesarias para la implementación
        # (relativas al cálculo de longitudes totales de cada documento, longitudes medias
        # para cada campo, y scorings previos de cada documento por su pagerank):
        self.lengths = {} # Document -> field -> length
        self.avg_length = {} # field -> length average
        self.pagerank_scores = {}

        # Cálculo inicial de las longitudes medias por campo (ver definición de método
        # calc_avg_length() justo a continuación):
        self.calc_avg_length()

    def calc_avg_length(self, debug=False):
        """ Computa las longitudes medias de cada campo en la colección.
            En el proceso rellena también el diccionario self.lengths
        """
        ### BEGIN YOUR CODE (FIXME)

        ### END YOUR CODE (FIXME)
        self.avg_length = {"title": avg_len_title, "headers": avg_len_headers,
                           "anchors": avg_len_anchors, "url": avg_len_url,
                           "body_hits": avg_len_body_hits}

    
    def normalize_doc_vec(self, q, d, doc_vec, debug=False):
        """ Normalizar las frecuencias crudas de los diferentes campos en el documento
            d usando la ecuación (1) especificada más arriba.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento a normalizar.
        Return:
            doc_vec (dict) : El vector de documento normalizado.
        """
        ### BEGIN YOUR CODE (FIXME)


        ### END YOUR CODE (FIXME)

        return doc_vec

    def get_net_vector(self, q, d):
        """ Obtener el vector neto global para el par (q,d), usando la ecuación (2) anterior.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            doc_vec (dict) : El vector de documento normalizado (ya sólo uno, incluyendo todos los términos incluídos en todos los campos).
        """
        ### BEGIN YOUR CODE (FIXME)

    
        ### END YOUR CODE (FIXME)

    def get_net_score(self, q, d):
        """ Obtener la puntuación global BM25F para el par (q,d), usando la ecuación (3) anterior.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            doc_vec (dict) : El scoring neto global, incluyendo ya también la puntuación por pagerank.
        """
        q_vec = self.get_query_vector(q)
        n_vec = self.get_net_vector(q, d)
        score = 0
        for term in n_vec.keys():
            if term in q_vec.keys():
                score += (n_vec[term]/(self.k1+n_vec[term])) * self.idf.get_idf(term)
        score += self.pagerank_lambda * math.log(self.pagerank_lambda_prime + d.pagerank)
        # print("PAGERANK:", d.pagerank)
        return score

Probamos la nueva clase _BM25FScorer_, primero con unos ciertos parámetros iniciales en los que hemos fijado $b_f=0 \quad \forall f$, así como $k_1=\lambda = \lambda'=0$, simplemente para depurar:

In [ ]:
# Imprimimos la consulta y el documento de ejemplo:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']
print("CONSULTA:", q,"\n")
print("DOCUMENTO:", d)

# Usamos consulta booleana, sin normalizar, e incluyendo en ella el IDF...
query_weight_scheme = {"tf": 'b', "df": 't', "norm": None}
# ... y con conteo de frecuencias crudas iniciales para el documento, normalizados por
# zonas de acuerdo a la ecuación (1):
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": "default"}

# Creamos el scorer BM25F con los anteriores parámetros, e inicialmente con los respectivos b_f
# inicializados a 0.0 para comprobar la corrección de los cálculos en los vectores separados
# por campos:
params_bm25f = {
    "url_weight" : 0.1,
    "title_weight": 0.15,
    "body_hits_weight" : 0.2,
    "header_weight" : 0.25,
    "anchor_weight" : 0.30,
    "b_url" : 0.0,
    "b_title" : 0.0,
    "b_header" : 0.0,
    "b_body_hits" : 0.0,
    "b_anchor" : 0.0,
    "k1": 0.0,
    "pagerank_lambda" : 0.0,
    "pagerank_lambda_prime" : 0.0,
}
bm25f_scorer = BM25FScorer(theIDF, query_dict, params_bm25f, query_weight_scheme, doc_weight_scheme)

print('\nVector de consulta:', bm25f_scorer.get_query_vector(q))
print('\nVector de documento:', bm25f_scorer.get_doc_vector(q, d))
print('\nVector neto:', bm25f_scorer.get_net_vector(q, d))
print('\nScoring neto:', bm25f_scorer.get_net_score(q, d))

assert bm25f_scorer.get_net_score(q, d)  == sum([theIDF.get_idf(term) if term in q else 0 for term, val in bm25f_scorer.get_net_vector(q, d).items()]), \
       "Fallo en chequeo de la clase BM25FScorer"

Si observamos la información impresa por la celda anterior, y prestamos atención al vector de documento impreso (separado por campos), comprobamos que los vectores resultantes coinciden con los de conteo originales (como corresponde a los valores $b_f=0 \quad \forall f$ usados para depurar en primera instancia).

Obsérvese también que el vector neto combina ya todos los términos en un sólo vector, según los pesos indicados en los parámetros usados (p.e., para el término _"events"_, que aparece con un valor de 1.0 tanto en el campo **url** como en el campo **title**, el valor es de 0.25, como corresponde a la suma $0.1*1.0+0.15*1.0 = 0.25$, con $w_u=0.1$ y $w_t=0.15$. El término _"aoerc"_, por su parte, aparece con un valor 1.4, correspondiente en este caso a la suma $0.2*7.0 = 1.4$, al aparecer únicamente en el campo **body_hits**, con $w_b=0.2$ y un conteo de apariciones de exactamente 7.0 en dicho campo.

Finalmente, puede comprobarse también que el _scoring_ neto obtenido coincide en este caso con la simple suma de los IDF de todos los términos incluidos en la consulta, como debe ser el caso al aplicar la ecuación (3) con $k_1=\lambda = \lambda'=0$.

Un ejercicio interesante es cambiar ahora los parámetros libres con otros valores con más sentido, para observar sus respectivas influencias en los resultados. Por ejemplo:

In [ ]:
params_bm25f = {
    "url_weight" : 0.1,
    "title_weight": 0.1,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "b_url" : 0.5,
    "b_title" : 0.5,
    "b_header" : 0.5,
    "b_body_hits" : 0.5,
    "b_anchor" : 0.5,
    "k1": 1.0,
    "pagerank_lambda" : 1.0,
    "pagerank_lambda_prime" : 2.0,
}
bm25f_scorer = BM25FScorer(theIDF, query_dict, params_bm25f, query_weight_scheme, doc_weight_scheme)

print('\nVector de consulta:', bm25f_scorer.get_query_vector(q))
print('\nVector de documento:', bm25f_scorer.get_doc_vector(q, d))
print('\nVector neto:', bm25f_scorer.get_net_vector(q, d))
print('\nScoring neto:', bm25f_scorer.get_net_score(q, d))

La salida sería la siguiente:

Es interesante reevaluar varias veces la celda anterior jugando con pequeños cambios aislados en los diferentes parámetros libres, e interpretar de esta forma su efecto inmediato tanto en los vectores de documento separados por campos como en el vector neto resultado, y el correspondiente scoring neto final.

# Función de ranking 3: ventana más pequeña

Finalmente, añadiremos la influencia de los tamaños de ventana al algoritmo BM25F. Para una consulta deda $q$, definimos la _ventana más pequeña_ $w_{q,d}$ como la secuencia más corta de tokens en el documento $d$ tal que todos los términos en la consulta $q$ están presentes en dicha secuencia. Una ventana sólo puede especificarse para un campo en particular, y para el caso concreto del campo _anchor\_text_, se exige que todos los términos en $q$ estén presentes en un enlace particular (esto es, si un término ocurre en el texto de un enlace, y otro en el de otro enlace diferente al mismo documento), no se considerará una misma ventana. Si, por otro lado, $d$ no contiene alguno de los términos de la consulta y, por tanto, no se puede encontrar dicha ventana, entonces definimos $w_{q,d} = \infty$.

Intuitivamente, cuanto más pequeña sea la ventana $w_{q,d}$, más relevante debería ser el documento $d$ para la consulta $q$. Por lo tanto, multiplicaremos el _scoring_ BM25F del documento por un factor de _boost_ basado en $w_{q,d}$, de forma que:

* Si $w_{q,d} = \infty$, entonces el factor de _boost_ es 1.0.
* Si $w_{q,d} = |Q|$, siendo $Q$ el número de términos únicos en $q$, entonces multiplicaremos el _scoring_ original por un factor predeterminado máximo $B$, estrictamente mayor que uno.
* Para valores de $w_{q,d}$ entre la longitud de la consulta e infinito, el factor de _boost_ deberá moverse entre B (valor máximo) y 1.0 (valor mínimo), decrementándose progresivamente conforme crece el tamaño de $w_{q,d}$.

Para esto último, podría aquí usarse un decrecimiento de tipo exponencial, o bien del tipo $\frac{1}{x}$. La siguiente gráfica ilustra una posible implementación de este último tipo de decrecimiento, para 4 valores diferentes de B:

In [ ]:
# Ilustramos cuatro funciones de decrecimiento basado en 1/x para los valores
# máximos de B = {4.0, 2.0, 1.75, 1.25}:
len_q_list = 10.0
max_win_len = 50
len_min = np.arange(len_q_list,max_win_len+1)
for B in [4.0, 2.0, 1.75, 1.25]:
    factor_win = 1.0+(B-1.0)*len_q_list/len_min
    plt.plot(len_min,factor_win,'+-',label=str(B));
plt.legend()

##  Clase _WindowScorer_

He aquí la definición de una clase _SmallestWindowScorer_ para implementar la técnica anterior. Se basa en la clase anterior _BM25Scorer_, ampliándola fundamentalmente con el método `get_boost_score`, que realiza el trabajo principal apoyándose a su vez en el método `min_sublist_with_all`. Éste último es el que en última instancia realiza la búsqueda efectiva de la ventana de texto más pequeña que contiene a todos los términos de la consulta:

In [ ]:
class SmallestWindowScorer(BM25FScorer):
    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None):
        super().__init__(idf, query_dict, params, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme) #modified
        # Añadimos el parámetro "B" (máximo boosting alcanzable):
        self.B = params["B"]

    
    def min_sublist_with_all(self, A, B):
        """ Calcula el tamaño de la mínima sublista de B que contiene a toda la
            lista de términos de A.
        Args:
            A (lista de términos) : Lista de términos en la consulta.
            B (lista de términos) : Lista de términos en la que realizar la
                                    búsqueda de la sublista más pequeña.
        Return:
            min_length (dict) : La longitud de la sublista más pequeña encontrada
                                (float('inf') si no existe tal sublista).
        """
        # BEGIN YOUR CODE
        

        ### END YOUR CODE
        # Devolvemos la longitud de la mínima sublista encontrada:
        return min_length
    

    def get_boost_score(self, q, d, debug=False):
        """ Calcula el factor de boost basado en la técnica 'smallest window'.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            debug (bool) : Flag para imprimir posible información de depuración.
        Return:
            factor_win (float) : Factor de boost, entre 1 y B.
        """

        # Lista de términos de la consulta de entrada:
        q_list = str(q).split()

        # Extraemos todas las listas de términos del documento, separadas
        # y procesadas debidamente según campos:
        all_lists = []
        try: # Campo url:
            d_url_list, _ = self.parse_url(d.url)
        except:
            d_url_list = []
        try: # Campo title:
            d_title_list = d.title.split()
        except:
            d_title_list = []
        try: # Campo headers:
            d_headers_lists = [h.split() for h in d.headers]
        except:
            d_headers_lists = []
        try: # Campo anchors:
            d_anchors_lists = [a.split() for a in d.anchors]
        except:
            d_anchors_lists = []
        try: # Campo body:
            # Construimos lista ficticia de términos a partir de body_hits,
            # rellenando con "-" donde no se conoce el término:
            max_pos = -1
            for k in d.body_hits.keys():
                max_pos_k = max(d.body_hits[k])
                if max_pos_k > max_pos:
                    max_pos = max_pos_k
            d_body_hits_list = (max_pos+1)*["-"]
            for k in d.body_hits.keys():
                for pos in d.body_hits[k]:
                    d_body_hits_list[pos] = k
        except:
            d_body_hits_list = []

        # Lista de todas las listas de términos a procesar para este documento:
        all_lists += [d_url_list] + [d_title_list] + d_headers_lists + d_anchors_lists + [d_body_hits_list]
        if debug:
            print(f"\nq_list: {q_list}")
            print("\nall_lists:")
            for l in all_lists:
                print(f" {l}")

        # Cómputo de la mínima ventana para todas las listas de todos los campos
        # (tamaño definitivo de la mínima ventana para este documento):
        if debug:
            print(f"\n Ternas (q_list, lista, min_dist):")
        len_min = float("inf")
        for i,lt in enumerate(all_lists):
            min_dist = self.min_sublist_with_all(q_list,lt)
            if debug:
               print(f"{q_list}     {lt}     {min_dist}")
            if min_dist < len_min:
                len_min = min_dist
        factor_win = 1.0+(self.B-1)*len(q_list)/len_min if len_min != float('inf') else 1.0
        return factor_win


    def get_net_score(self, q, d):
        """ Obtener el scoring neto para un par consulta - documento utilizando
            un factor de boosting computado usando la similaridad por la técnica
            del mínimo tamaño de ventana.
        Args:
            d (Document) : El documento.
            q (Query) : La consulta.

        Return:
            El scoring crudo multiplicado por el factor de boost.
        """
        boost = self.get_boost_score(q, d)
        raw_score = super().get_net_score(q, d)
        return boost * raw_score

Probamos en la celda siguiente la clase anterior. Definimos unos parámetros por defecto para la clase (incluyendo un $B$ máximo de 2.0), elegimos una consulta _q_ y un documento _d_ de prueba, y calculamos un factor de _boost_ en modo `debug=True`, para comprobar la corrección de los cómputos intermedios para calcularlo:

In [ ]:
# Parámetros para la creación de la clase:
params_window = {
    "B": 2.0,
    "url_weight" : 1.0,
    "title_weight": 0.1,
    "body_hits_weight" : 0.25,
    "header_weight" : 0.5,
    "anchor_weight" : 0.3,
    "b_url" : 0.0,
    "b_title" : 0.0,
    "b_header" : 0.0,
    "b_body_hits" : 0.0,
    "b_anchor" : 0.0,
    "k1": 2.0,
    "pagerank_lambda" : 0.1,
    "pagerank_lambda_prime" : 1.0,
}

# Consulta y documento de prueba:
q = Query("stanford parking")
d = query_dict[q]["https://transportation.stanford.edu/"]

# Definición de instancia de la clase:
smallest_window_scorer = SmallestWindowScorer(theIDF, query_dict, params_window)

# Prueba de funcionamiento interno del método get_boost_score(...):
smallest_window_scorer.get_boost_score(q, d, debug=True)

# Prueba del método get_net_score(...) que calcula el scoring neto:
print(f"\nScoring neto tras usar el factor de boost: {smallest_window_scorer.get_net_score(q, d):5.3f}")

# Ranking del dataset

## Clase _Rank_

Definimos una sencilla clase conteniendo sólo métodos de clase, que agrupa varias funciones de utilidad en la construcción de _rankings_ de documentos resultado de la búsqueda para una determinada consulta:

In [ ]:
class Rank:
    # Sólo métodos de clase:
    def score(query_dict, score_type, idf, params):
        """ Llamar a esta función para puntuar (y ordenar según esta puntuación)
            todos los documentos correspondientes a una consulta, en un conjunto
            completo (dado en forma de mapping consultas -> {documentos}).
        Args:
            query_dict (dict) :  Mapeo Query->url->Document.
            score_type (str) : Tipo de scorer a usar ("baseline", "cosine", "bm25f", "window").
            idf (dict) : Diccionario IDF.
            params(dict) : Parámetros para el scorer usado.
        Return
            query_rankings (dict) : Un mapeo Query->Document->(r,s) (r=ranking=entero, comenzando en 1; s=score=float).
        """
        # Seleccionar subclase concreta de AbstractScorer para crear el tipo de instancia
        # concreta de scorer a utilizar:
        if score_type == "baseline": scorer = BaselineScorer(idf)
        elif score_type == "cosine": scorer = CosineSimilarityScorer(idf, query_dict, params)
        elif score_type == "bm25f": scorer = BM25FScorer(idf, query_dict, params)
        elif score_type == "window": scorer = SmallestWindowScorer(idf, query_dict, params)
        else: print('Tipo erróneo de scorer (debe ser "baseline", "cosine", "bm25f" o "window")!')

        # Diccionario donde se almacenará el mapping consultas->rankings devuelto:
        query_rankings = {} # query -> rank
        # Bucle que recorre todas las consultas en el diccionario de entrada:
        for i, query in enumerate(query_dict.keys()):
            q = query
            
            ### BEGIN YOUR CODE (FIXME)
            # Bucle que recorre todos los urls para cada consulta, obteniendo el score correspondiente:
           
            # Ordenamos los documentos por sus scorings...
            
            # ... y asignamos el mapeo Document->(ranking,score) resultante de todos los documentos
            # a la consulta correspondiente en el mapeo de salida Query->Document->ranking:
            
            ### END YOUR CODE (FIXME)

        return query_rankings

    def write_ranking_to_file(query_rankings, ranked_result_filename):
        """ Función que exporta los rankings obtenidos sobre un dataset de
           consultas-documentos a un fichero de texto.
        Args:
            query_rankings (dict) : Un mapeo Query->Document->ranking (ranking=entero, comenzando en 1).
            ranked_result_filename (str): Ruta al archivo de salida.
        """
        with open(ranked_result_filename, "w") as f:
            for query, docs in query_rankings.items():
                f.write("query: "+ query.__str__() + "\n")
                for doc, rank in docs.items():
                    output_info = "  url: " + doc.url + "\n" + \
                                  "    title: " + doc.title + "\n" + \
                                  "    rank:  " + str(rank[0]) + "\n" + \
                                  "    score: " + str(rank[1]) + "\n"
                    f.write(output_info)
        print(f"¡Escritura de archivo {ranked_result_filename} realizada!")

## Generación de archivos de rankings

Usando la clase `Rank` anterior, realizamos las ordenaciones de todas las consultas contenidas en `query_dict` (provenientes del archivo de _training_ inicial). Realizamos cuatro _rankings_ usando las cuatro técnicas desarrolladas (_"baseline"_, _"cosine"_, _"bm25f"_, _"window"_), guardando cada una de ellas en el correspondiente archivo `output/ranked_train_{tecnica}.txt`:

In [ ]:
for method, params in zip(["baseline", "cosine", "bm25f", "window"], [None, params_cosine, params_bm25f, params_window]):
    query_dict = load_train_data(os.path.join(data_dir, "pa3.signal.train"))
    query_rankings = Rank.score(query_dict, method, theIDF, params)
    Rank.write_ranking_to_file(query_rankings, os.path.join("output", "ranked_train_"+method+".txt"))
    print(f"Rankings realizados para {len(query_rankings)} consultas, (usando el {method} scorer)\n")

A título de ejemplo de los resultados obtenidos, a continuación mostramos los _rankings_ realizados por los cuatro métodos para los diez documentos obtenidos para la primera consulta del _dataset_ de _training_:

In [ ]:
!echo RANKING 1ª CONSULTA, BASELINE:
!head -41 output/ranked_train_baseline.txt
!echo

!echo RANKING 1ª CONSULTA, COSINE:
!head -41 output/ranked_train_cosine.txt
!echo

!echo RANKING 1ª CONSULTA, BM25F:
!head -41 output/ranked_train_bm25f.txt
!echo

!echo RANKING 1ª CONSULTA, WINDOW:
!head -41 output/ranked_train_window.txt
!echo
